# DSAA-Kulimi Rwanda Data Camp Capstone Project <br> Lockdowns Impact on Air Quality 🌍

GitHub repo: https://github.com/stoufa/Lockdowns-Impact-on-Air-Quality

In [ ]:
# Enabling Line Wrapping
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Getting Data

In [ ]:
# creates the project's folder structure if it doesn't already exist
!mkdir -p data/{air_quality,lockdowns}
!mkdir -p data/air_quality/{raw,clean}
!mkdir -p data/lockdowns/{raw,clean}

## Lockdown Dates Data

We'll be getting this data from the [COVID-19 Lockdown dates by country Kaggle dataset](https://www.kaggle.com/jcyzag/covid19-lockdown-dates-by-country)

In [ ]:
from pathlib import Path
import pandas as pd
from pprint import pprint as pp

In [ ]:
# better way to display pandas dataframes
%load_ext google.colab.data_table

In [ ]:
DATA_FOLDER = Path('data')

AIR_QUALITY_DATA_FOLDER = DATA_FOLDER / 'air_quality'
AIR_QUALITY_RAW_DATA_FOLDER = AIR_QUALITY_DATA_FOLDER / 'raw'
AIR_QUALITY_CLEAN_DATA_FOLDER = AIR_QUALITY_DATA_FOLDER / 'clean'

LOCKDOWNS_DATA_FOLDER = DATA_FOLDER / 'lockdowns'
LOCKDOWNS_RAW_DATA_FOLDER = LOCKDOWNS_DATA_FOLDER / 'raw'
LOCKDOWNS_CLEAN_DATA_FOLDER = LOCKDOWNS_DATA_FOLDER / 'clean'

In [ ]:
# download data
DATASET_URL = 'https://www.dropbox.com/s/1kc7imm6dvzv9jv/COVID-19%20Lockdown%20dates%20by%20country.zip'
!wget {DATASET_URL} &> /dev/null

In [ ]:
!unzip "COVID-19 Lockdown dates by country.zip"

Archive:  COVID-19 Lockdown dates by country.zip
  inflating: countryLockdowndates.csv  
  inflating: countryLockdowndatesJHUMatch.csv  


In [ ]:
# move the data to its appropriate location
!mv *.csv data/lockdowns/raw/

In [ ]:
# remove all CSV files in data/
# !rm data/*.csv

let's now take a look at the data

In [ ]:
# display the first 10 lines of the data file (with line numbers)
CSV_FILE_PATH = LOCKDOWNS_RAW_DATA_FOLDER / 'countryLockdowndates.csv'
!head {CSV_FILE_PATH} | cat -n

     1	Country/Region,Province,Date,Type,Reference
     2	Afghanistan,,24/03/2020,Full,https://www.thestatesman.com/world/afghan-govt-imposes-lockdown-coronavirus-cases-increase-15-1502870945.html
     3	Albania,,08/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Albania
     4	Algeria,,24/03/2020,Full,https://www.garda.com/crisis24/news-alerts/325896/algeria-government-implements-lockdown-and-curfew-in-blida-and-algiers-march-23-update-7
     5	Andorra,,16/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Andorra
     6	Angola,,24/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Angola
     7	Antigua and Barbuda,,,None,
     8	Argentina,,20/03/2020,Full,https://www.bloomberg.com/news/articles/2020-03-20/argentina-orders-exceptional-lockdown-in-bid-to-contain-virus
     9	Armenia,,24/03/2020,Full,https://www.azatutyun.am/a/30506939.html
    10	Australia,Australian Capital Territory,,None,https://en.wikipedia.org/wiki/

We can see that we have a comma-separated dataset with 5 columns


In [ ]:
df = pd.read_csv(CSV_FILE_PATH)

In [ ]:
# let's lowercase colum names to make using them easier later
df.columns = map(lambda x : x.lower(), df.columns)
# let's also replace slashes '/' with underscores '_' for the same reason
df.columns = map(lambda x : x.replace('/', '_'), df.columns)
df.columns

Index(['country_region', 'province', 'date', 'type', 'reference'], dtype='object')

In [ ]:
# displays the first 5 rows of the data
df.head()

,country_region,province,date,type,reference
0,Afghanistan,NaN,24/03/2020,Full,https://www.thestatesman.com/world/afghan-govt...
1,Albania,NaN,08/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
2,Algeria,NaN,24/03/2020,Full,https://www.garda.com/crisis24/news-alerts/325...
3,Andorra,NaN,16/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
4,Angola,NaN,24/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...


In [ ]:
# getting more details about the column names and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   country_region  307 non-null    object
 1   province        129 non-null    object
 2   date            230 non-null    object
 3   type            307 non-null    object
 4   reference       238 non-null    object
dtypes: object(5)
memory usage: 12.1+ KB


In [ ]:
# getting a summary of the main descriptive statistics of this data file
df.describe()

,country_region,province,date,type,reference
count,307,129,230,307,238
unique,182,129,37,3,146
top,US,Hong Kong,23/01/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
freq,54,1,31,207,40


In [ ]:
# find rows having null date values
df[df[['date']].isnull().any(axis=1)]
# we can notice that we have 77 entries without dates, so,
# we can remove them, and we'll see if we can find better data elsewhere

,country_region,province,date,type,reference
5,Antigua and Barbuda,NaN,NaN,None,NaN
8,Australia,Australian Capital Territory,NaN,None,https://en.wikipedia.org/wiki/2020_coronavirus...
9,Australia,New South Wales,NaN,None,https://en.wikipedia.org/wiki/2020_coronavirus...
10,Australia,Northern Territory,NaN,None,https://en.wikipedia.org/wiki/2020_coronavirus...
11,Australia,Queensland,NaN,None,https://en.wikipedia.org/wiki/2020_coronavirus...
...,...,...,...,...,...
288,US,South Carolina,NaN,None,NaN
289,US,South Dakota,NaN,None,NaN
292,US,Utah,NaN,None,NaN
299,US,Wyoming,NaN,None,NaN


In [ ]:
df = df.dropna(subset=['date'])
df

,country_region,province,date,type,reference
0,Afghanistan,NaN,24/03/2020,Full,https://www.thestatesman.com/world/afghan-govt...
1,Albania,NaN,08/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
2,Algeria,NaN,24/03/2020,Full,https://www.garda.com/crisis24/news-alerts/325...
3,Andorra,NaN,16/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
4,Angola,NaN,24/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
...,...,...,...,...,...
301,Vatican City,NaN,10/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
302,Venezuela,NaN,16/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
303,Vietnam,NaN,19/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
304,West Bank and Gaza,NaN,05/03/2020,Full,NaN


In [ ]:
# let's change the date format to make it easier to use
# now, it's `dd-mm-yyyy`, so, let's convert it to `yyyy-mm-dd`
# e.g. 16/03/2020 --> 2020-03-16

from datetime import datetime

datetime.strptime('16/03/2020', '%d/%m/%Y').strftime('%Y-%m-%d')

'2020-03-16'

In [ ]:
# check if date column have any null values
df.date.isnull().values.any()

False

In [ ]:
df.date = df.date.apply(
    lambda x : datetime.strptime(x, '%d/%m/%Y').strftime('%Y-%m-%d')
)
df.date

0      2020-03-24
1      2020-03-08
2      2020-03-24
3      2020-03-16
4      2020-03-24
          ...    
301    2020-03-10
302    2020-03-16
303    2020-03-19
304    2020-03-05
306    2020-03-27
Name: date, Length: 230, dtype: object

In [ ]:
df.sort_values(by='date', ascending=True)

,country_region,province,date,type,reference
81,China,Tianjin,2020-01-23,Full,https://en.wikipedia.org/wiki/2020_Hubei_lockd...
84,China,Yunnan,2020-01-23,Full,https://en.wikipedia.org/wiki/2020_Hubei_lockd...
83,China,Xinjiang,2020-01-23,Full,https://en.wikipedia.org/wiki/2020_Hubei_lockd...
53,China,Anhui,2020-01-23,Full,https://en.wikipedia.org/wiki/2020_Hubei_lockd...
54,China,Beijing,2020-01-23,Full,https://en.wikipedia.org/wiki/2020_Hubei_lockd...
...,...,...,...,...,...
285,US,Pennsylvania,2020-04-02,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
275,US,Nevada,2020-04-02,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
271,US,Mississippi,2020-04-03,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
246,US,Alabama,2020-04-04,Full,https://en.wikipedia.org/wiki/2020_coronavirus...


In [ ]:
print(sorted(df.date.unique()))

['2020-01-23', '2020-01-26', '2020-02-02', '2020-02-23', '2020-02-25', '2020-02-27', '2020-03-02', '2020-03-05', '2020-03-06', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-06']


In [ ]:
# date range
df.date.min(), df.date.max()

('2020-01-23', '2020-04-06')

In [ ]:
# countries/regions found in the dataset
print(sorted(df.country_region.unique()))

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belgium', 'Belize', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Canada', 'Chile', 'China', 'Colombia', 'Congo (Kinshasa)', 'Costa Rica', 'Cote dIvoire', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, South', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritius', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'N

In [ ]:
# provinces found in the dataset
# dropna() is used to remove null values
print(sorted(df.province.dropna().unique()))

['Alabama', 'Alaska', 'Alberta', 'Anguilla', 'Anhui', 'Arizona', 'Aruba', 'Beijing', 'British Columbia', 'California', 'Chongqing', 'Colorado', 'Connecticut', 'Delaware', 'Faroe Islands', 'Florida', 'French Polynesia', 'Fujian', 'Gansu', 'Gibraltar', 'Guam', 'Guangdong', 'Guangxi', 'Guizhou', 'Hainan', 'Hawaii', 'Hebei', 'Heilongjiang', 'Henan', 'Hong Kong', 'Hubei', 'Hunan', 'Idaho', 'Illinois', 'Indiana', 'Inner Mongolia', 'Isle of Man', 'Jiangsu', 'Jiangxi', 'Jilin', 'Kansas', 'Kentucky', 'Liaoning', 'Louisiana', 'Macau', 'Maine', 'Manitoba', 'Martinique', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nevada', 'New Brunswick', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'Newfoundland and Labrador', 'Ningxia', 'North Carolina', 'Northwest Territories', 'Nova Scotia', 'Ohio', 'Ontario', 'Oregon', 'Pennsylvania', 'Prince Edward Island', 'Puerto Rico', 'Qinghai', 'Quebec', 'Rhode Island', 'Saskatchewan', 'Shaanxi', 'Shandong', 

In [ ]:
# types of lockdown found in the dataset
print(sorted(df.type.unique()))

['Full', 'Partial']


In [ ]:
# number of duplicated rows
df.duplicated(keep='first').sum()

0

In [ ]:
# country names are not standard, let's use country codes instead
# we'll need them when we want to merge this dataset with other ones

# download data
COUNTRY_CODE_DATA_URL = 'https://www.dropbox.com/s/h1u9kkc1l30lw8a/country_codes_and_flags.tsv'
!wget {COUNTRY_CODE_DATA_URL} &> /dev/null

In [ ]:
# move the TSV file to the raw/ folder
!mv *.tsv data/lockdowns/raw

In [ ]:
# display the first 10 lines of the data file (with line numbers)
TSV_FILE_PATH = LOCKDOWNS_RAW_DATA_FOLDER / 'country_codes_and_flags.tsv'
!head {TSV_FILE_PATH} | cat -n

     1	Country Name	Alpha-2 Code	Alpha-3 Code	Numeric Code	Flag
     2	Afghanistan	AF	AFG	4	🇦🇫
     3	Åland Islands	AX	ALA	248	🇦🇽
     4	Albania	AL	ALB	8	🇦🇱
     5	Algeria	DZ	DZA	12	🇩🇿
     6	American Samoa	AS	ASM	16	🇦🇸
     7	Andorra	AD	AND	20	🇦🇩
     8	Angola	AO	AGO	24	🇦🇴
     9	Anguilla	AI	AIA	660	🇦🇮
    10	Antarctica	AQ	ATA	10	🇦🇶


In [ ]:
df2 = pd.read_csv(TSV_FILE_PATH, sep='\t')
df2

,Country Name,Alpha-2 Code,Alpha-3 Code,Numeric Code,Flag
0,Afghanistan,AF,AFG,4,🇦🇫
1,Åland Islands,AX,ALA,248,🇦🇽
2,Albania,AL,ALB,8,🇦🇱
3,Algeria,DZ,DZA,12,🇩🇿
4,American Samoa,AS,ASM,16,🇦🇸
...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,🇼🇫
245,Western Sahara,EH,ESH,732,🇪🇭
246,Yemen,YE,YEM,887,🇾🇪
247,Zambia,ZM,ZMB,894,🇿🇲


In [ ]:
# let's lowercase colum names to make using them easier later
df2.columns = map(lambda x : x.lower(), df2.columns)
# let's also replace spaces ' ' and dashes '-' with underscores '_' for the same reason
df2.columns = map(lambda x : x.replace(' ', '_'), df2.columns)
df2.columns = map(lambda x : x.replace('-', '_'), df2.columns)
list(df2.columns)

['country_name', 'alpha_2_code', 'alpha_3_code', 'numeric_code', 'flag']

In [ ]:
# now we have to check if all the countries in our dataset have an entry
# in the country_codes table
dataset_countries = df.country_region.unique()
cc_countries = df2.country_name.unique() # country codes countries

In [ ]:
type(dataset_countries), type(cc_countries)

(numpy.ndarray, numpy.ndarray)

In [ ]:
# these country names are in our dataset but we couldn't find them
# in the country codes table, to fix this, we add them manually
set(dataset_countries) - set(cc_countries)

{'Bolivia',
 'Congo (Kinshasa)',
 'Cote dIvoire',
 'Iran',
 'Korea, South',
 'Kosovo',
 'Moldova',
 'Palestine',
 'Russia',
 'Syria',
 'Taiwan*',
 'US',
 'United Kingdom',
 'Vatican City',
 'Venezuela',
 'Vietnam',
 'West Bank and Gaza'}

In [ ]:
# for 'Bolivia' for example, we can notice that it's called
# 'Bolivia (Plurinational State of)' in the country codes table
# this is why we should work with country codes instead! because country
# names can differ from one dataset to another!
# the solution here is to update the country_name column to suit our needs
# in this project

df2.loc[df2.alpha_2_code == 'BO']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
26,Bolivia (Plurinational State of),BO,BOL,68,🇧🇴


In [ ]:
# Congo (Kinshasa) --> Democratic Republic of the Congo (CD)
# https://en.wikipedia.org/wiki/Democratic_Republic_of_the_Congo
# df2[df2.country_name.str.contains('Congo')]
df2[df2.country_name.str.contains('Congo, Democratic')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
51,"Congo, Democratic Republic of the",CD,COD,180,🇨🇩


In [ ]:
df2[df2.country_name.str.contains('Ivoire')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
54,Côte d'Ivoire,CI,CIV,384,🇨🇮


In [ ]:
df2[df2.country_name.str.contains('Iran')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
105,Iran (Islamic Republic of),IR,IRN,364,🇮🇷


In [ ]:
# South Korea --> Republic Korea
# https://en.wikipedia.org/wiki/South_Korea
# df2[df2.country_name.str.contains('Korea')]
df2[df2.country_name.str.contains('Korea, Republic')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
119,"Korea, Republic of",KR,KOR,410,🇰🇷


In [ ]:
# Kosovo is not found in our table, so, we have to add this row manually!
# https://en.wikipedia.org/wiki/Kosovo
# df2[df2.country_name.str.contains('Kosovo')]
df2[df2.alpha_2_code == 'XK']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_name  249 non-null    object
 1   alpha_2_code  248 non-null    object
 2   alpha_3_code  249 non-null    object
 3   numeric_code  249 non-null    int64 
 4   flag          249 non-null    object
dtypes: int64(1), object(4)
memory usage: 9.9+ KB


In [ ]:
# 'country_name', 'alpha_2_code', 'alpha_3_code', 'numeric_code', 'flag'
# string,         string,         string,         int,            string
# https://en.wikipedia.org/wiki/Kosovo
# https://laendercode.net/en/2-letter-code/xk
# https://www.dnb.com/content/dam/english/dnb-solutions/sales-and-marketing/iso_3digit_numeric_country_codes.xls
# https://emojipedia.org/flag-kosovo/

df2 = df2.append({
    'country_name': 'Kosovo',
    'alpha_2_code': 'XK',
    'alpha_3_code': 'XXK',
    'numeric_code': 926,
    'flag': '🇽🇰'
}, ignore_index=True)

In [ ]:
# double-checking that 'Kosovo' now exists in the country codes table
df2[df2.alpha_2_code == 'XK']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
249,Kosovo,XK,XXK,926,🇽🇰


In [ ]:
df2[df2.country_name.str.contains('Moldova')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
145,"Moldova, Republic of",MD,MDA,498,🇲🇩


In [ ]:
df2[df2.country_name.str.contains('Palestine')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
170,"Palestine, State of",PS,PSE,275,🇵🇸


In [ ]:
df2[df2.country_name.str.contains('Russia')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
183,Russian Federation,RU,RUS,643,🇷🇺


In [ ]:
df2[df2.country_name.str.contains('Syria')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
216,Syrian Arab Republic,SY,SYR,760,🇸🇾


In [ ]:
df2[df2.country_name.str.contains('Taiwan')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
217,"Taiwan, Province of China",TW,TWN,158,🇹🇼


In [ ]:
df2[df2.country_name.str.contains('United States')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
235,United States Minor Outlying Islands,UM,UMI,581,🇺🇲
236,United States of America,US,USA,840,🇺🇸


In [ ]:
df2[df2.country_name.str.contains('United Kingdom')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
234,United Kingdom of Great Britain and Northern I...,GB,GBR,826,🇬🇧


In [ ]:
# Vatican City also is not found in the country codes table, so,
# we should add it manually
df2[df2.country_name.str.contains('Vatican')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag


In [ ]:
# https://en.wikipedia.org/wiki/Vatican_City
# https://wits.worldbank.org/wits/wits/witshelp/content/codes/country_codes.htm
# https://emojipedia.org/flag-vatican-city/

df2 = df2.append({
    'country_name': 'Vatican City',
    'alpha_2_code': 'VA',
    'alpha_3_code': 'VAT',
    'numeric_code': 336,
    'flag': '🇻🇦'
}, ignore_index=True)

In [ ]:
# double-checking that 'Vatican City' now exists in the country codes table
# p.s. also, we can notice that the country actually exists with another name
# 'Holy See', yet another proof on why country codes are more reliable
df2[df2.alpha_2_code == 'VA']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
98,Holy See,VA,VAT,336,🇻🇦
250,Vatican City,VA,VAT,336,🇻🇦


In [ ]:
# let's remove the duplicated row
df2 = df2[df2.country_name != 'Holy See']

In [ ]:
# double-checking that the duplicated row is gone!
df2[df2.country_name == 'Holy See']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag


In [ ]:
df2[df2.country_name.str.contains('Venezuela')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
240,Venezuela (Bolivarian Republic of),VE,VEN,862,🇻🇪


In [ ]:
df2[df2.country_name.str.contains('Vietnam')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag


In [ ]:
# https://en.wikipedia.org/wiki/Vietnam
# here, we can notice that 'Vietnam' is spelled 'Viet Nam' in the country
# codes table (on two words instead of one)
df2[df2.alpha_2_code == 'VN']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
241,Viet Nam,VN,VNM,704,🇻🇳


In [ ]:
df2[df2.country_name.str.contains('Gaza')]

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag


In [ ]:
# https://en.wikipedia.org/wiki/Gaza_Strip
# https://en.wikipedia.org/wiki/Palestinian_territories
df2[df2.alpha_2_code == 'PS']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
170,"Palestine, State of",PS,PSE,275,🇵🇸


In [ ]:
# I'm not sure why in this dataset, 'Palestine' and 'West Bank and Gaza'
# are not considered the same! anyways, we should duplicate the row
# corresponding to the country code 'PS' in the country codes table

df2[df2.alpha_2_code == 'PS'].to_dict(orient='records')

[{'alpha_2_code': 'PS',
  'alpha_3_code': 'PSE',
  'country_name': 'Palestine, State of',
  'flag': '🇵🇸',
  'numeric_code': 275}]

In [ ]:
df2 = df2.append({
    'country_name': 'West Bank and Gaza',
    'alpha_2_code': 'PS',
    'alpha_3_code': 'PSE',
    'numeric_code': 275,
    'flag': '🇵🇸'
}, ignore_index=True)

In [ ]:
# and let's change 'Palestine' manually as well
df2.country_name = df2.country_name.apply(
    lambda x : 'Palestine' if x == 'Palestine, State of' else x)

In [ ]:
# making sure that the name was changed successfully
df2[df2.country_name == 'Palestine']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
169,Palestine,PS,PSE,275,🇵🇸


In [ ]:
# double-checking that the 'PS' country code have 2 entries
# one for 'Palestine' and another for 'West Bank and Gaza'
df2[df2.alpha_2_code == 'PS']

,country_name,alpha_2_code,alpha_3_code,numeric_code,flag
169,Palestine,PS,PSE,275,🇵🇸
250,West Bank and Gaza,PS,PSE,275,🇵🇸


In [ ]:
new_countries = {
    'Bolivia':            'BO',
    'Congo (Kinshasa)':   'CD',
    'Cote dIvoire':       'CI',
    'Iran':               'IR',
    'Korea, South':       'KR',
    'Kosovo':             'XK',
    'Moldova':            'MD',
    # 'Palestine':          'PS', # special case: added manually in the country codes table
    'Russia':             'RU',
    'Syria':              'SY',
    'Taiwan*':            'TW',
    'US':                 'US',
    'United Kingdom':     'GB',
    'Vatican City':       'VA',
    'Venezuela':          'VE',
    'Vietnam':            'VN',
    # 'West Bank and Gaza': 'PS' # special case: added manually in the country codes table
}

In [ ]:
# making sure that each country code have a single entry in the table
for country_name, country_alpha2_code in new_countries.items():
  match = df2[df2.alpha_2_code == country_alpha2_code].to_dict(orient='records')
  n_results = len(match)
  # print(type(match)) # <class 'list'>
  # pp(match)
  # break
  assert n_results == 1, f'{country_alpha2_code} have more than an entry!'
print('✅ all good!')

✅ all good!


In [ ]:
# updating the country names to fit the ones we're using in this dataset
for country_name, country_alpha2_code in new_countries.items():
  match = df2[df2.alpha_2_code == country_alpha2_code].to_dict(orient='records')[0]
  print(f'change {match.get("country_name"):60} to \t {country_name}')

change Bolivia (Plurinational State of)                             to 	 Bolivia
change Congo, Democratic Republic of the                            to 	 Congo (Kinshasa)
change Côte d'Ivoire                                                to 	 Cote dIvoire
change Iran (Islamic Republic of)                                   to 	 Iran
change Korea, Republic of                                           to 	 Korea, South
change Kosovo                                                       to 	 Kosovo
change Moldova, Republic of                                         to 	 Moldova
change Russian Federation                                           to 	 Russia
change Syrian Arab Republic                                         to 	 Syria
change Taiwan, Province of China                                    to 	 Taiwan*
change United States of America                                     to 	 US
change United Kingdom of Great Britain and Northern Ireland         to 	 United Kingdom
change Vatican

In [ ]:
# just to make sure we're doing the right thing, let's copy the dataframe
df2_test = df2.copy(deep=True)

In [ ]:
# getting the index of a row in the dataframe
row_index = df2[df2.alpha_2_code == 'BO'].index.tolist()[0]
row_index, type(row_index)

(26, int)

In [ ]:
# update row at given index
# dataframe.at[index, 'column-name'] = 'new value'
df2.at[26, 'country_name']

'Bolivia (Plurinational State of)'

In [ ]:
for country_name, country_alpha2_code in new_countries.items():
  row = df2_test[df2_test.alpha_2_code == country_alpha2_code]
  row_index = row.index.tolist()[0]
  old_value = df2_test.at[row_index, 'country_name']
  # print('row_index', row_index)
  print(f'(# {row_index:3}) change {old_value:60} to \t {country_name}')

(#  26) change Bolivia (Plurinational State of)                             to 	 Bolivia
(#  51) change Congo, Democratic Republic of the                            to 	 Congo (Kinshasa)
(#  54) change Côte d'Ivoire                                                to 	 Cote dIvoire
(# 104) change Iran (Islamic Republic of)                                   to 	 Iran
(# 118) change Korea, Republic of                                           to 	 Korea, South
(# 248) change Kosovo                                                       to 	 Kosovo
(# 144) change Moldova, Republic of                                         to 	 Moldova
(# 182) change Russian Federation                                           to 	 Russia
(# 215) change Syrian Arab Republic                                         to 	 Syria
(# 216) change Taiwan, Province of China                                    to 	 Taiwan*
(# 235) change United States of America                                     to 	 US
(# 233) change

In [ ]:
# updating country names (on the testing copy)
for country_name, country_alpha2_code in new_countries.items():
  row = df2_test[df2_test.alpha_2_code == country_alpha2_code]
  row_index = row.index.tolist()[0]
  df2_test.at[row_index, 'country_name'] = country_name

In [ ]:
# let's check again if all the countries in our dataset have an entry
# in the country_codes table
dataset_countries = df.country_region.unique()
cc_countries_ = df2_test.country_name.unique() # country codes countries
set(dataset_countries) - set(cc_countries_)

set()

perfect! empty set returned, this means that all of our country names now can be found in the country codes table.  
let's now add the country codes column to our dataset

In [ ]:
# updating country names (on the original copy)
for country_name, country_alpha2_code in new_countries.items():
  row = df2[df2.alpha_2_code == country_alpha2_code]
  row_index = row.index.tolist()[0]
  df2.at[row_index, 'country_name'] = country_name

In [ ]:
dataset_countries = df.country_region.unique()
cc_countries = df2.country_name.unique() # country codes countries
set(dataset_countries) - set(cc_countries)

set()

In [ ]:
# deleting the testing copy
del df2_test

In [ ]:
# let's merge the dataframes, to do that, let's take a second look
# on the column names in each one of them
df.columns.tolist(), df2.columns.tolist()

(['country_region', 'province', 'date', 'type', 'reference'],
 ['country_name', 'alpha_2_code', 'alpha_3_code', 'numeric_code', 'flag'])

In [ ]:
merged_df = df.merge(df2, how='inner', left_on='country_region', right_on='country_name')
columns_to_drop = ['reference', 'country_name', 'alpha_3_code', 'numeric_code', 'flag']
merged_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
merged_df

,country_region,province,date,type,alpha_2_code
0,Afghanistan,NaN,2020-03-24,Full,AF
1,Albania,NaN,2020-03-08,Full,AL
2,Algeria,NaN,2020-03-24,Full,DZ
3,Andorra,NaN,2020-03-16,Full,AD
4,Angola,NaN,2020-03-24,Full,AO
...,...,...,...,...,...
225,Vatican City,NaN,2020-03-10,Full,VA
226,Venezuela,NaN,2020-03-16,Full,VE
227,Vietnam,NaN,2020-03-19,Full,VN
228,West Bank and Gaza,NaN,2020-03-05,Full,PS


In [ ]:
# making sure that the dataset have the same number of rows before
# and after merging it with the country codes table
n_rows_before = len(df.index)
n_rows_after = len(merged_df.index)
n_rows_before == n_rows_after

True

In [ ]:
# update column names (alpha_2_code -> country_code)
merged_df.columns = ['country_region', 'province', 'date', 'type', 'country_code']

In [ ]:
# reorder the columns: let's put country_code right after the country/region name
# date here refers to starting date
new_columns_order = ['country_region', 'country_code', 'province', 'date', 'type']
merged_df = merged_df[new_columns_order]

In [ ]:
# save the processed version of the data
TEST_LOCKDOWNS_DATA_FILE_DST = LOCKDOWNS_CLEAN_DATA_FOLDER / 'country_lockdown_dates.csv'
merged_df.to_csv(TEST_LOCKDOWNS_DATA_FILE_DST, index=False, header=True)

# References
* [python - How to convert a date string to different format - Stack Overflow](https://stackoverflow.com/questions/14524322/how-to-convert-a-date-string-to-different-format)
* [python - Filter pandas DataFrame by substring critera - Stack Overflow](https://stackoverflow.com/questions/11350770/filter-pandas-dataframe-by-substring-critera)


---